# Project #2 Artificial intelligance(Genetics)
# (spring 2022)

    Amirmahdi Ansaripour

    Student ID: 810198358

### Approach

Implementing an encryption method using genetic algorithm

### Description

An encoded file is given to us and we're to decode it using an 14-length key and a table.
The table is simply a 26 * 26 matrix, which the first row is [A,B,....,Z] and next rows are left-shifted of their previous row.(for example, second row is [B,C,...,Z,A])

If we can find the key, decoding won't be difficult anymore. Since we don't have any information about the key and should begin completely random, GA algorithm seems to be useful. 

### Defining parameters

Firstly, we define some variables which aren't obtained during execution and should be initialized manully by user.

Besides, some helpful packages and libraries are imported.


In [17]:
import numpy as np
import string
import random
from random import choice
import math
ALPHABET_SIZE = 26
POP_SIZE = 100
ALPHABETS = list(string.ascii_uppercase)
CROSS_OVER_PROB = 0.7
MUTATION_PROB = 0.05
CARRIED_ON_PARENTS = math.floor(0.48 * POP_SIZE)
KEY_LENGTH = 14
INPUT2 = "encoded_text.txt"
INPUT1 = "global_text.txt"
ZERO = 0

### Making encryption table
As mentioned, a table is needed to decode the text which is simply implemented by shiftinig each row to left and place it in the next row



In [18]:
 def makeEncryptionTable(self):
    self.table = np.array([ALPHABETS] * ALPHABET_SIZE)
    shift = -1
    for i in range(1, ALPHABET_SIZE):
        self.table[i] = np.roll(self.table[i], shift)
        shift = shift - 1

### Cleaning data and making dictionary
The first step is to clean the global text given to the decoder and making a dictionary of valid words. This dictionary is useful when tokenizing the encoded text.

The way I used to tokenize the global text:

Each non-alphabet letter (such as numbers, dots, commas, ...) is a delimeter that seperates two valid words. The words are added to the dictionary.

In [19]:
def makeDictionary(self, globalTxt):
    meanings = set()
    token = ""
    for i in range(len(globalTxt)):
        if(globalTxt[i].isalpha()):
            if(globalTxt[i] != ""):
                token += globalTxt[i].lower()
                if(i == len(globalTxt) - 1):
                    meanings.add(token)
        else:
            if(token != ""):
                meanings.add(token)
                token = ""
    return list(sorted(meanings))

### Defining initial concepts
Each chromosome is a string of length 14 (mentioned in assignment). Hence, each gene is a letter of that string. The definition is flexible enough that accepts different randomizing functions such as mutation, crossover, etc.

The desired solution is a chromosome which decodes the encrypted text to a meaningful one (using the dictionary and the table).

### Generating initial population
To start the algorithm, we need an absolute-random population. Population size is a parameter that is assigned by the user (and has a very significant impact on the algorithm). 

In [20]:
def initPopulation(self, Popsize):
    lst = []
    for i in range(Popsize):
        newChrome = []
        for j in range(self.keyLength):
            newChrome.append(choice(ALPHABETS))
        lst.append(newChrome)
    return lst

### Implementing crossover and mutation
The main reason of using a GA is the two functions implemented below. They're used for converging the population to a specific area or diverging if needed.
Each function has a probability for its own. It's used to determine wheter the function should be run or not.

In [21]:
def crossOver(self, chrom1, chrom2, prob):
    child = []
    for i in range(self.keyLength):
        probable = random.random()
        if(probable < prob):
            child.append(chrom1[i])
        else:
            child.append(chrom2[i])
    return child

In [22]:
def mutation(self, chrom, prob):
    for i in range(len(chrom)):
        probable = random.random()
        if(probable < prob):
            chrom[i] = choice(ALPHABETS)
    return chrom


### Designating fitness function
Accorging to the definition of chromosome and gene, fitness function won't be difficult any more. Encrypted text will be decoded using the code and number of valid words is the fitness value. Here comes the dictionary that we've defined it recently. 

In [23]:
def matchWithTable(self, chrom):
    ## Consider two issues : lower/upper case + ascii equvalent of chars
    chromCounter = 0
    token = ""
    toBeMeant = []
    
    self.oneToOne.clear()
    enc = ""
    
    translated = ""
    for i in range(len(self.encoded)):
        if(self.encoded[i].isalpha()):
            enc += self.encoded[i]
            isLow = self.encoded[i].islower()
            upper = self.encoded[i].upper()
            ascii_chrom = ord(chrom[chromCounter]) - 65
            ascii_upper = ord(upper) - 65
            chromCounter += 1
            intersect = self.table[ascii_chrom][ascii_upper]    
            if(isLow):
                intersect = intersect.lower()
            token += intersect.lower()
            translated += intersect
            if(i == len(self.encoded) - 1):
                toBeMeant.append(token)
                self.oneToOne[token] = enc
            if(chromCounter == KEY_LENGTH):
                chromCounter = 0
        else:
            if(token != ""):      ## like ; , . ()
                toBeMeant.append(token)
            self.oneToOne[token] = enc
            token = ""
            enc = ""
            translated += self.encoded[i]   ## add non-alphas exactly to last string
            
    return [toBeMeant, translated]

In [24]:
def search(self,x):
    if x in self.dictionary:
        return True
    else:
        return False

def fitness(self, chrom):
    matched = self.matchWithTable(chrom)
    cost = 0
    unmatched = []
    for i in range(len(matched[0])):
        if(self.search(matched[0][i])):
            continue
        else:
            unmatched.append(matched[0][i])
            cost += 1    
    if (cost == 0):
        self.finalTxt = matched[1]
    return cost


In [25]:
def calcCromsandCost(self):
    chromsWithCosts = []
    for i in range(len(self.chroms)):
        cost = self.fitness(self.chroms[i])
        chromsWithCosts.append((cost, self.chroms[i]))
    chromsWithCosts.sort()
    return chromsWithCosts

In [26]:
def finalize(self, chrom):
    if(chrom[0] == 0):
        self.foundSolution = True
        self.solution = chrom[1]
    

In [27]:
# def generateParanormally(self, cost):
#     newCost = cost
#     print("Paranormally")
#     for i in range(len(self.chroms)):
#         for j in range(i + 1, len(self.chroms)):
#             chrom1 = self.chroms[i]
#             chrom2 = self.chroms[j]
#             child1 = self.crossOver(chrom1, chrom2, self.crossOverProb)
#             m = self.calcCost(child1)
#             newCost1 = m[0]
#             child2 = self.mutation(self.chroms[i], 1)
#             child3 = self.mutation(self.chroms[j], 1)
#             newCost2 = self.calcCost(child2)[0]
#             newCost3 = self.calcCost(child3)[0]
#             newCost = min(newCost1, newCost2, newCost3)
#             if(newCost < cost):
#                 print("Finally:", newCost)
                
    
#     print("Failure:", newCost)


In [28]:
def crossandMutation(self, nextGeneration, carriedOnChroms):
    while(len(nextGeneration) < self.popSize):
        chrom1 = []
        chrom2 = []
        chrom1,chrom2 = random.sample(carriedOnChroms, 2)
        
        child = self.crossOver(chrom1, chrom2, self.crossOverProb)
        nextGeneration.append(child)
        child2 = child
        child2 = self.mutation(child2, self.mutateProb)
        if(len(nextGeneration) < self.popSize):
            nextGeneration.append(child2)
    return nextGeneration

### Generating next population
After processing all the elements in current population, and not finding the solution between them, it's time to generate the next population. Used methods are crossover, mutation, and carrying some well-fitted chromosomes directly to next stage.

In [29]:
def generate(self, chromsWithCosts):
    carriedOnChroms = []
    nextGeneration = []
    for i in range(self.holdChroms):
        nextGeneration.append(chromsWithCosts[i][1])
        carriedOnChroms.append(chromsWithCosts[i][1])
    nextGeneration = self.crossandMutation(nextGeneration, carriedOnChroms)
    return nextGeneration

In [30]:
def newPopulation(self):
    chromsWithCosts = self.calcCromsandCost()
            
#     print("Fitness : ", chromsWithCosts[0][0])
    
    self.finalize(chromsWithCosts[0])    
    
    nextGeneration = self.generate(chromsWithCosts)

    return nextGeneration

### Final algorithm
After explaining all details, the algorithm is made by putting them together. It ends when the solution is found and the final encoded text is returned.

In each iteration, a new population is generated and processed.

In [31]:
def decode(self):
    self.makeEncryptionTable()
    self.chroms = self.initPopulation(self.popSize)
    while(not(self.foundSolution)):
        res = self.newPopulation()
        self.chroms = res
#         print("Best current chromosome : ", self.chroms[0])
    return self.finalTxt

In [32]:
class Decoder:
    def __init__(self, globalTxt, encodedTxt, keyLength_):
        self.keyLength = keyLength_
        self.popSize = POP_SIZE  #constant size of population
        self.chroms = []
        self.table = []
        self.crossOverProb = CROSS_OVER_PROB
        self.mutateProb = MUTATION_PROB
        self.foundSolution = False
        self.oneToOne = {}
        self.solution = []
        self.holdChroms = CARRIED_ON_PARENTS
        self.encoded = encodedTxt
        self.changePattern = False
        self.dictionary = self.makeDictionary(globalTxt)
        self.finalTxt = ""
        
    makeDictionary = makeDictionary
    makeEncryptionTable = makeEncryptionTable
    initPopulation = initPopulation
    crossOver = crossOver
    mutation = mutation
    matchWithTable = matchWithTable
    newPopulation = newPopulation    
    decode = decode
    fitness = fitness
    search = search
    calcCromsandCost = calcCromsandCost
    finalize = finalize
    generate = generate
    crossandMutation = crossandMutation
    
globalText = open(INPUT1).read()   
encodedText = open(INPUT2).read()
D = Decoder(globalText, encodedText, KEY_LENGTH)
decodedText = D.decode()
print(decodedText)

Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probable—through the work of Joliot in France as well as Fermi and Szilard in America—that it may become possible to set up a nuclear chain reaction in a large mass of uranium by which vast amounts of power and large quantities of new radium-like elements would be generated. Now it appears almost certai

## Questions


##### Important points:
1)Since most stages of algorithm are done completely random, these  answers are generally true and you may test a GA with a random population that doesn't accept the answers.

2)In each step, next population is made up of a percentage of best-fitting chromosomes in addition to their random offsprings (mostly crossovered and some mutated). 


    1) Name some problems of starting a genetic algorithm with an under-crowded population, what about an over-crowded one?
Over-Crowded: Although the chance of reaching the goal chromosome is higher in this case, but we should consider the time spent for calculating the fitness of all chromosomes and generating their offsprings when they are alot.
Besides, when the number of chromosomes is extremely high, the algorithm may not converge early.

Under-Crowded: When the first population size is extremely low, we should'nt expect that the final chromosome will appear in our population very soon, due to the limited states of 
combining the parents.

In other words, the process of making a new generation may not take alot of time, but it should be repeated more frequently.

What's more, when the size of a low-populated sample doesn't change during the algorithm, chance of getting stuck and staying in local optimas gets higher.


    2) If the population size increases in each step, how does this increasing impact speed and accuracy of algorithm?

If the size increases, it means that more best-fitting 
elements and their offsprings are produced and the probabilty that our desired answer is among them increases. In other words, number of iterations of making new population decreases but processing each population takes more time. So there isn't an exact yes/no answer for speed part and highly depends on the implementation.

About the accuracy, it increases or remains unchanged. Because more offsprings of well-fitted parents (hopefully, with better fitness points) are added. But if we're stuck in a local optima, probably increasing population won't do anything useful.


    3) Why does GA use both crossover and mutation methods? What will happen if we use only one of them?
Crossover is for convergence and mutation is for divergence of the population. Both are essential based on the following reasons:

a) If crossover is omitted, the population doesn't converge to a specific well-fitted chromosome and some fixed relatively good chromosomes will always appear in the sample and become useless.

b) Mutation helps us to move to a different area of solutions if we're converged to and stuck in a local optima and there isn't hope to get better answers. In other words, it helps to diverge the population from it's current state. So if we put it aside, we will probably waste a lot of time in local optimas.

    4) Which operation has a more positive influence on accuracy? Which one impacts faster?
Both crossover and mutation are useful for getting more closed to the global optima in each stage. But as their definition suggests, convergence to an optima (local or global) is the result of crossover. So it seems to affect accuracy better and faster than mutation.

    5) Using mutation and crossover, it's possible that after a few steps, the population is still unchanged. How this problem can be solved?

Applying n-point crossover with a high n parameter, and increasing the mutation and crossover probabilities are helpful to change the chromosomes (Note that if mutation probability gets one, chromosomes and respectively the population will difinitely change)

    6) If using only one of the methods was allowed, which one would be more helpful?

Without crossover, the population doesn't converge to a soultion gradually and acts haphazardly. So that the algorithm converts into randomized local search. So it's more necessary than mutation. 

    7) What techniques can be used in this problem to reach the desired result faster?
When calculating fitness, binary search can be used for finding the decoded tokens in the dictionary(sorting the dictionary is done as a preprocess before decoding)

Some kinds of parallelism can also be used when tokenizing the dictionary or the encoded text or finding fitnesses


### Conclusion
So far, we've learnt the concept of genetic algorithms and when they are used and for which kind of problems. Besides, some techniques were mentioned to enhance the algorithm. Our learning passed throught the way of a decoding method. 

### Refrences

1.https://www.researchgate.net/post/How_to_avoid_repetition_in_the_populations_of_the_Genetic_Algorithm

2.https://www.geeksforgeeks.org/ml-convergence-of-genetic-algorithms/

3.https://www.geeksforgeeks.org/genetic-algorithms/